In [1]:
from math import sqrt
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import norm

In [2]:
import numpy as np
from scipy.stats import norm

S0 = 100
K = 108
T = 1
num_steps = 100
sigma = 0.2
mu = 0.08
r = 0.04
n_path = 10_000


def gbm(S0, r, sigma, t, n_path):
    drift = r * t
    time_steps = np.diff(t, prepend=0)
    normals = np.random.standard_normal((n_path, num_steps+1))
    normals = np.vstack([normals, -normals])
    vol = sigma * np.sqrt(time_steps) * normals.cumsum(axis=1)
    exponent = drift + vol
    paths = S0 * np.exp(exponent)
    return paths

t = np.linspace(0, T, num_steps + 1)
S = gbm(S0, r, sigma, t, n_path)
S

array([[100.        ,  99.90348836, 102.04874514, ..., 100.21177683,
        100.93417203, 101.45191309],
       [100.        , 104.26218151, 104.00601309, ..., 135.9183546 ,
        136.76381076, 138.60256962],
       [100.        , 103.94244679, 100.27563826, ...,  86.90094602,
         84.00166575,  85.45834438],
       ...,
       [100.        , 103.91727522, 104.0522162 , ..., 131.79138726,
        130.48755072, 133.43778456],
       [100.        , 100.07988076, 103.47169808, ..., 124.98912128,
        123.51090654, 123.92570218],
       [100.        ,  97.45735845,  99.18463405, ..., 158.83364315,
        160.30203371, 157.08193191]])

In [3]:
def _d1(S, K, r, sigma, t, T):
    t2m = T - t
    numerator = np.log(S / K) + (r + 0.5 * sigma**2) * t2m
    denominator = sigma * np.sqrt(t2m)
    with np.errstate(divide='ignore'):
        return numerator / denominator

def _d2(d1, sigma, t, T):
    t2m = T - t
    return d1 - sigma * np.sqrt(t2m)

def call(S, K, r, t, T, d1, d2):
    return S * norm.cdf(d1) - K * np.exp(-r * (T - t)) * norm.cdf(d2)

def put(S, K, r, t, T, d1, d2):
    return -S * norm.cdf(-d1) + K * np.exp(-r * (T - t)) * norm.cdf(-d2)

def call_delta(d1):
    return norm.cdf(d1)

def put_delta(d1):
    return norm.cdf(d1) - 1

d1 = _d1(S, K, r, sigma, t, T)
d2 = _d2(d1, sigma, t, T)

c = call(S, K, r, t, T, d1, d2)
p = put(S, K, r, t, T, d1, d2)

In [4]:
print(f'Call={c[0, 0]}')
print(f'Put={p[0, 0]}')

Call=6.370611880864999
Put=10.135871309315903


In [5]:
delta = call_delta(d1)[:, :-1]
delta

array([[4.66208127e-01, 4.62912144e-01, 5.04270361e-01, ...,
        3.06553574e-02, 4.61033740e-03, 3.99509711e-04],
       [4.66208127e-01, 5.48351062e-01, 5.42470413e-01, ...,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
       [4.66208127e-01, 5.42233521e-01, 4.68983858e-01, ...,
        4.10944578e-11, 1.06528859e-14, 2.41000381e-36],
       ...,
       [4.66208127e-01, 5.41750666e-01, 5.43360148e-01, ...,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
       [4.66208127e-01, 4.66434797e-01, 5.32137870e-01, ...,
        9.99999288e-01, 9.99999904e-01, 1.00000000e+00],
       [4.66208127e-01, 4.13842120e-01, 4.47065799e-01, ...,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00]])

In [6]:
discount_factors = np.exp(-r * t)
txns = np.diff(delta, axis=1, prepend=0, append=0) * S
txns[:, -1] += np.maximum(S[:, -1] - K, 0)
disounted_txns = discount_factors * txns
costs = disounted_txns.sum(axis=1)
np.mean(costs)

6.410150292120234

In [7]:
discount_factors = np.exp(-r * t)
txns = np.diff(delta - 1, axis=1, prepend=0, append=0) * S
txns[:, -1] += np.maximum(K - S[:, -1], 0)
disounted_txns = discount_factors * txns
costs = disounted_txns.sum(axis=1)
np.mean(costs)

10.175409720571142